# Resample hourly to daily

This notebook is used for resampling the restacked 20km outputs to daily files. 

We will submit a slurm job for a subset of the hourly files, that will then be resampled to a daily resolution.

In [1]:
import xarray as xr
from config import *
import luts
import slurm

Iterate over the output variable names, i.e. the resampled/aggregated variable names, which may be the same as the WRF variable name, and write the sbatch scripts:

In [2]:
out_varnames = luts.resample_varnames
years = luts.groups[group]["years"]
year_str = f"{years[0]}-{years[-1]}"

sbatch_fps = []
for varname in out_varnames:
    wrf_varname = luts.resample_varnames[varname]["wrf_varname"]
    # write to .slurm script
    sbatch_fp = slurm_dir.joinpath(f"resample_{group}_{year_str}_{varname}.slurm")
    # filepath for slurm stdout
    sbatch_out_fp = slurm_dir.joinpath(f"resample_{group}_{year_str}_{varname}_%j.out")
    sbatch_head = slurm.make_sbatch_head(
        slurm_email, partition, conda_init_script
    )

    args = {
        "sbatch_fp": sbatch_fp,
        "sbatch_out_fp": sbatch_out_fp,
        "resample_script": resample_script,
        "hourly_dir": hourly_dir,
        "daily_dir": daily_dir,
        "wrf_varname": wrf_varname,
        "out_varname": varname,
        "aggr": luts.resample_varnames[varname]["aggr"],
        "fn_str": luts.groups[group]["fn_str"],
        "year_str": year_str,
        "ncpus": ncpus,
        "sbatch_head": sbatch_head,
    }

    slurm.write_sbatch_resample(**args)
    sbatch_fps.append(sbatch_fp)

Remove existing slurm output scripts if you fancy it:

In [3]:
for varname in out_varnames:
    _ = [fp.unlink() for fp in list(slurm_dir.glob(f"resample_{group}_{year_str}_{varname}_*.out"))]

Submit the `.slurm` scripts with `sbatch`:

In [4]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]

This should complete this step of the pipeline. Once the slurm jobs have all finished, proceed to the quality control notebook. 